In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Thêm đường dẫn đến thư mục src để import các module
sys.path.append(os.path.abspath(os.path.join('..')))

from src import data_processing as dp
from src import visualization as viz

# Cấu hình để hiển thị biểu đồ trong notebook
%matplotlib inline

# Tải dữ liệu
filepath = '../data/raw/aug_train.csv'
data = dp.load_data(filepath)

# 1. Tổng Quan về Dữ Liệu (Dataset Overview)

### 1.1. Thông Tin Cơ Bản (Basic Information)

In [ ]:
basic_info = dp.get_basic_info(data)
print(f"Số dòng: {basic_info['shape'][0]}")
print(f"Số cột: {basic_info['shape'][1]}")
print(f"Kích thước tổng thể: {basic_info['memory_usage'] / 1e6:.2f} MB")

**Mỗi dòng đại diện cho điều gì?**

**Mỗi dòng trong tập dữ liệu này đại diện cho một ứng viên đã đăng ký tham gia chương trình đào tạo của công ty.**
Dòng dữ liệu mô tả đầy đủ hồ sơ của từng ứng viên, bao gồm thông tin nhân khẩu học, trình độ học vấn, kinh nghiệm làm việc, tình trạng công việc hiện tại, số giờ đào tạo và một biến mục tiêu cho biết họ có đang tìm kiếm cơ hội việc làm mới hay có khả năng gắn bó làm việc cho công ty sau khi hoàn thành khóa đào tạo.


In [ ]:
# Hiển thị 5 dòng đầu tiên
print("5 dòng đầu tiên:")
print(data[:5])

# Hiển thị 5 dòng cuối cùng
print("\n5 dòng cuối cùng:")
print(data[-5:])

### 1.2. Tính Toàn Vẹn của Dữ Liệu (Data Integrity)

In [ ]:
duplicate_info = dp.check_duplicates(data)
print(f"Số dòng trùng lặp (không tính enrollee_id): {duplicate_info['duplicated_rows']}")
print(f"Tỷ lệ trùng lặp: {duplicate_info['duplicated_percentage']:.2f}%")

In [ ]:
# Hiển thị các dòng trùng lặp nếu có
if duplicate_info['duplicated_rows'] > 0:
    print("\nCác dòng được xác định là trùng lặp:")
    for duplicate in duplicate_info['representative_duplicates']:
        print(duplicate['row'])
        print(f"Count: {duplicate['count']}")
        print()
else:
    print("\nKhông tìm thấy dòng nào trùng lặp.")

**Xử lý dữ liệu trùng lặp:**

Các dòng trùng lặp **nên được loại bỏ**.

Toàn bộ 49 bản ghi trùng đều xuất hiện **dưới dạng cặp giống hệt nhau trên tất cả các thuộc tính trừ `enrollee_id`**. Điều này cho thấy những bản ghi này thực chất mô tả **cùng một ứng viên** được ghi nhận nhiều lần, không mang thêm thông tin mới.

Tỷ lệ trùng lặp tuy nhỏ (**0.26%**) nhưng việc loại bỏ vẫn cần thiết để đảm bảo dữ liệu **sạch**, tránh tác động tiêu cực đến mô hình dự báo.

In [ ]:
# Có bao nhiêu dòng hoàn toàn đủ/trống thông tin
row_completeness = dp.count_row_completeness(data)
print(f"Số dòng đủ hết thông tin: {row_completeness['complete_rows']}")
print(f"Số dòng thiếu hết thông tin: {row_completeness['empty_rows']}")

### 1.3. Kiểm Kê Cột và Kiểu Dữ Liệu (Column Inventory & Data Types)

**Ý nghĩa của từng cột:**

- **enrollee_id**: ID duy nhất cho ứng viên.
- **city**: Mã thành phố.
- **city_development_index**: Chỉ số phát triển của thành phố (đã được scale).
- **gender**: Giới tính của ứng viên.
- **relevent_experience**: Kinh nghiệm liên quan của ứng viên.
- **enrolled_university**: Loại khóa học đại học đã đăng ký (nếu có).
- **education_level**: Trình độ học vấn của ứng viên.
- **major_discipline**: Chuyên ngành học chính của ứng viên.
- **experience**: Tổng số năm kinh nghiệm.
- **company_size**: Số lượng nhân viên trong công ty của nhà tuyển dụng hiện tại.
- **company_type**: Loại hình của nhà tuyển dụng hiện tại.
- **last_new_job**: Chênh lệch số năm giữa công việc trước đó và công việc hiện tại.
- **training_hours**: Số giờ đào tạo đã hoàn thành.
- **target**: Biến mục tiêu.
  - `0`: Không tìm kiếm sự thay đổi công việc.
  - `1`: Đang tìm kiếm sự thay đổi công việc.

**Các cột có liên quan và các cột cần loại bỏ:**

- **Cột cần loại bỏ:** `enrollee_id` là một mã định danh duy nhất và không có giá trị dự đoán. Nó nên được loại bỏ khỏi quá trình huấn luyện mô hình.
- **Các cột liên quan:** Tất cả các cột khác đều có khả năng chứa thông tin hữu ích để dự đoán biến mục tiêu `target`.

In [ ]:
print("Kiểu dữ liệu của các cột:")
for name, dtype in basic_info['dtypes'].items():
    print(f"- {name}: {dtype}")

**Ghi chú kiểu dữ liệu `<U...`:**

Khi đọc dữ liệu từ file CSV bằng **NumPy** (ví dụ dùng `np.loadtxt` hoặc `np.genfromtxt`), các cột chứa dữ liệu dạng chữ sẽ được NumPy chuyển thành kiểu **chuỗi Unicode** với định dạng **`<U...`**. Ký hiệu này thể hiện kiểu *Unicode string* có độ dài cố định, trong đó số phía sau (ví dụ `<U8`, `<U23`, ...) là **độ dài tối đa của chuỗi trong cột**. Khi NumPy gặp cột chứa ký tự, nó không có kiểu linh hoạt như `object` của pandas, do đó nó buộc phải chọn một kiểu chuỗi Unicode có độ dài đủ lớn để chứa giá trị dài nhất trong cột. 

**Đánh giá kiểu dữ liệu:**

Một số cột đang được NumPy đọc dưới dạng **chuỗi Unicode (`<U…`)**, mặc dù bản chất của chúng là **biến số** hoặc **biến thứ bậc (ordinal)**. Điều này khiến chúng mang kiểu dữ liệu không phù hợp và cần chuyển đổi lại trước khi phân tích hoặc đưa vào mô hình.


**Các cột *cần chuyển kiểu dữ liệu*:**

**1. experience (`<U3`)**

* Giá trị có dạng: `<1`, `>20`, `15`, `5`, …
* Đây là dữ liệu **số hoặc thứ bậc**, nhưng bị đọc thành chuỗi.

**2. company_size (`<U9`)**

* Dạng: `50-99`, `100-500`, `1000-4999`
* Bản chất là **ordinal**, có thể mã hóa thành số hoặc thứ bậc.

**3. last_new_job (`<U5`)**

* Dạng: `never`, `1`, `2`, `>4`
* Là dữ liệu **thứ bậc theo thời gian**.

**4. target (`float64`)**

* Hiện tại: `0.0` và `1.0`
* Đây là **nhãn nhị phân**, nên để dạng `int` mới đúng bản chất.

## 2. Phân Tích Giá Trị Thiếu (Missing Data Analysis)

### 2.1. Đánh Giá Tổng Quan (Overall Assessment)

In [ ]:
missing_summary = dp.get_missing_summary(data)
print("Tóm tắt giá trị thiếu theo từng cột:")
for col, summary in missing_summary.items():
    if summary['missing_count'] > 0:
        print(f"- Cột '{col}': {summary['missing_count']} giá trị thiếu ({summary['missing_percentage']:.2f}%)")

In [ ]:
viz.plot_missing_summary(missing_summary)

In [ ]:
viz.plot_missing_value_heatmap(data)

**Có phải các giá trị thiếu là ngẫu nhiên không?**

Heatmap và biểu đồ cột cho thấy các giá trị thiếu tập trung ở một số cột nhất định, đặc biệt là `gender`, `enrolled_university`, `education_level`, `major_discipline`, `experience`, `company_size`, `company_type`, và `last_new_job`. Việc thiếu dữ liệu có vẻ không hoàn toàn ngẫu nhiên. Ví dụ, `company_size` và `company_type` thường bị thiếu cùng nhau. Để xác định xem có một khuôn mẫu nào không, chúng ta sẽ kiểm tra mối quan hệ giữa việc thiếu dữ liệu và biến mục tiêu.

In [ ]:
cols_to_compare = [col for col, summary in missing_summary.items() if 0 < summary['missing_percentage'] < 50]
for col in cols_to_compare:
    missing_comparison = dp.compare_target_rate_by_missing(data, col)
    viz.plot_target_rate_by_missing(missing_comparison, col)

**Nhận xét:**

Phân tích tỷ lệ `target=1` (tìm kiếm việc mới) giữa nhóm có và không có giá trị thiếu cho thấy những khác biệt đáng chú ý:

**Các biến có sự khác biệt lớn:**
- **company_type**: Nhóm có giá trị thiếu có tỷ lệ target=1 cao hơn đáng kể (0.39 so với 0.18), chênh lệch gấp hơn 2 lần. Điều này cho thấy việc thiếu thông tin về loại hình công ty có thể liên quan chặt chẽ đến xu hướng tìm việc mới.
- **company_size**: Tương tự, nhóm thiếu dữ liệu có tỷ lệ target=1 cao hơn (0.41 so với 0.18), thể hiện mối liên hệ mạnh mẽ giữa việc không cung cấp thông tin quy mô công ty và ý định chuyển việc.

**Kết luận quan trọng:**
Bản thân việc thiếu dữ liệu, đặc biệt ở các biến liên quan đến thông tin công ty hiện tại (company_type, company_size), là một tín hiệu dự báo mạnh mẽ cho xu hướng tìm kiếm việc làm mới. Điều này gợi ý rằng thay vì đơn thuần xử lý missing values bằng imputation, nên tạo thêm các biến nhị phân (indicator variables) để đánh dấu trạng thái thiếu dữ liệu, giúp mô hình học được mẫu hình này.

### 2.2. Chiến Lược cho Từng Cột (Per Column Strategy)

**Tại sao các giá trị có thể bị thiếu?**

- **Thông tin không áp dụng:** Ví dụ, một người chưa bao giờ đi làm sẽ không có `company_size` hoặc `company_type`.
- **Từ chối trả lời:** Ứng viên có thể không muốn chia sẻ thông tin nhạy cảm như `gender` hoặc chi tiết về công ty hiện tại.
- **Lỗi thu thập dữ liệu:** Có thể có lỗi trong quá trình khảo sát hoặc nhập liệu.

**Kế hoạch xử lý:**

Phân tích ở trên cho thấy một insight quan trọng: **bản thân việc thiếu dữ liệu là một tín hiệu dự báo mạnh mẽ**. 

Do đó, chiến lược tốt nhất không phải là điền giá trị thiếu (imputation) một cách đơn giản, vì điều đó sẽ làm mất đi thông tin quý giá này. Thay vào đó, chúng ta nên:
1.  **Tạo một loại (category) riêng:** Đối với các biến phân loại, chúng ta sẽ coi giá trị thiếu là một loại mới, ví dụ: 'Not Specified' hoặc 'Missing'.
2.  **Sử dụng các mô hình có khả năng xử lý giá trị thiếu:** Các mô hình dựa trên cây quyết định như LightGBM hoặc XGBoost có thể xử lý các giá trị thiếu một cách tự nhiên.
3.  **Tạo biến chỉ báo (Indicator Variables):** Đối với mỗi cột có giá trị thiếu, chúng ta có thể tạo một cột nhị phân mới (`<column_name>_is_missing`) để cho mô hình biết rõ ràng rằng giá trị đó đã bị thiếu.

## 3. Phân Tích Đơn Biến (Univariate Analysis)

### 3.1. Phân Tích Biến Mục Tiêu (Target Variable Analysis)

In [ ]:
target_dist = dp.get_target_distribution(data)
viz.plot_target_distribution(target_dist['counts'], target_dist['percentages'])

**Nhận xét:**

Tập dữ liệu có sự **mất cân bằng nghiêm trọng** với lớp `target=0` (không muốn chuyển việc) chiếm đa số (khoảng 75%) so với lớp `target=1` (muốn chuyển việc, khoảng 25%). Tỷ lệ này là khoảng 3:1. Điều này cần được xử lý trong giai đoạn mô hình hóa.

### 3.2. Phân Tích Các Biến Số (Numerical Columns Analysis)

In [ ]:
numerical_cols = [name for name, dtype in data.dtype.fields.items() if np.issubdtype(dtype[0], np.number)]
numerical_cols.remove('enrollee_id')
if 'target' in numerical_cols:
    numerical_cols.remove('target')
print(f"Các biến số ({len(numerical_cols)}): {numerical_cols}")

#### Phân Phối & Xu Hướng Trung Tâm (Distribution & Central Tendency)

In [ ]:
numerical_summary = dp.get_numerical_summary(data, numerical_cols)
for col, summary in numerical_summary.items():
    print(f"Thống kê mô tả cho cột '{col}':")
    for stat, value in summary.items():
        print(f"  {stat:<10} {value:>10.2f}")
    print("\n")

In [ ]:
for col in numerical_cols:
    viz.plot_numerical_distribution_and_boxplot(data, col)

**Nhận xét:**
- **`city_development_index`**: Phân phối bị lệch trái (left-skewed), cho thấy hầu hết các ứng viên đến từ các thành phố có chỉ số phát triển cao. Điều này là hợp lý.
- **`training_hours`**: Phân phối bị lệch phải (right-skewed), cho thấy đa số ứng viên chỉ tham gia các khóa đào tạo ngắn giờ.

#### Miền Giá Trị & Ngoại Lệ (Range & Outliers)

**Đánh giá miền giá trị và ngoại lệ:**

- **`city_development_index`**: Miền giá trị từ `0.45` đến `0.95`. Các giá trị này hợp lý vì chỉ số phát triển thường được chuẩn hóa trong khoảng [0, 1]. Các giá trị thấp (outliers) có thể là các ứng viên từ các thành phố rất kém phát triển, đây là thông tin thật và hữu ích.
- **`training_hours`**: Miền giá trị từ `1` đến `336`. Các giá trị này có vẻ hợp lý. Các giá trị ngoại lệ (outliers) ở mức cao có thể là những ứng viên rất chăm chỉ hoặc tham gia các khóa học dài hạn. Đây là các giá trị cực trị hợp lệ, không phải lỗi nhập liệu.

#### Chất Lượng Dữ Liệu (Data Quality)

In [ ]:
print("Kiểm tra các giá trị không thể có:")
print(f"Số giờ đào tạo <= 0: {np.sum(data['training_hours'] <= 0)}")
print(f"Chỉ số phát triển thành phố < 0: {np.sum(data['city_development_index'] < 0)}")
print(f"Chỉ số phát triển thành phố > 1: {np.sum(data['city_development_index'] > 1)}")

**Nhận xét:**
Không có giá trị bất khả thi nào được tìm thấy trong các cột số. Dữ liệu có vẻ sạch về mặt này.

### 3.3. Phân Tích Các Biến Phân Loại (Categorical Columns Analysis)

In [ ]:
categorical_cols = [name for name, dtype in data.dtype.fields.items() if not np.issubdtype(dtype[0], np.number)]
print(f"Các biến phân loại ({len(categorical_cols)}): {categorical_cols}")

#### Chất Lượng Dữ Liệu (Data Quality)

In [ ]:
print("Kiểm tra tính nhất quán của các giá trị phân loại:")
for col in categorical_cols:
    # Bỏ qua cột city vì có quá nhiều giá trị
    if col == 'city': continue
    unique_vals = np.unique(data[col])
    print(f"- Cột '{col}' ({len(unique_vals)} giá trị duy nhất): {unique_vals}")

**Nhận xét:**

Việc kiểm tra sơ bộ các giá trị duy nhất cho thấy:
- **Không có sự thiếu nhất quán rõ ràng:** Không có các lỗi chính tả rõ ràng hoặc các biến thể của cùng một giá trị (ví dụ: 'Male' và 'male').
- **Giá trị rỗng:** Các giá trị rỗng (`''`) đã được xác định trong phần phân tích giá trị thiếu.
- **Giá trị 'Other':** Một số cột như `gender` và `company_type` có giá trị 'Other'. Đây là một giá trị hợp lệ.

Nhìn chung, chất lượng dữ liệu của các biến phân loại khá tốt.

#### Phân Phối Giá Trị (Value Distribution)

##### city

In [ ]:
print(f"Cột 'city' có {len(np.unique(data['city']))} giá trị duy nhất (cardinality cao).")
print("Phân tích chi tiết cho cột này sẽ được thực hiện riêng nếu cần thiết, vì trực quan hóa tất cả các thành phố sẽ không hiệu quả.")

##### gender

In [ ]:
viz.plot_categorical_analysis(data, 'gender')

##### relevent_experience

In [ ]:
viz.plot_categorical_analysis(data, 'relevent_experience')

##### enrolled_university

In [ ]:
viz.plot_categorical_analysis(data, 'enrolled_university')

##### education_level

In [ ]:
viz.plot_categorical_analysis(data, 'education_level')

##### major_discipline

In [ ]:
viz.plot_categorical_analysis(data, 'major_discipline')

##### experience

In [ ]:
viz.plot_categorical_analysis(data, 'experience')

##### company_size

In [ ]:
viz.plot_categorical_analysis(data, 'company_size')

##### company_type

In [ ]:
viz.plot_categorical_analysis(data, 'company_type')

##### last_new_job

In [ ]:
viz.plot_categorical_analysis(data, 'last_new_job')

**Nhận xét tổng quan về các đặc trưng phân loại:**

Phân tích chi tiết từng biến cho thấy các yếu tố như `relevent_experience`, `enrolled_university`, `experience`, và `company_size` có mối liên hệ rõ ràng với biến mục tiêu. Các biến này cho thấy sự khác biệt đáng kể về tỷ lệ `target=1` giữa các nhóm khác nhau, làm cho chúng trở thành những ứng cử viên mạnh mẽ cho mô hình dự đoán.

## 4. Phân Tích Đa Biến (Bivariate/Multivariate Analysis)

### 4.1. Mối Quan Hệ & Tương Quan (Relationships & Correlations)

#### Tương quan giữa các biến số

In [ ]:
viz.plot_correlation_heatmap(data, numerical_cols)

**Nhận xét:**

Heatmap tương quan cho thấy không có mối tương quan tuyến tính mạnh giữa `city_development_index` và `training_hours`. Điều này cho thấy chúng cung cấp thông tin độc lập cho mô hình.

#### Tương quan giữa Biến số và Biến mục tiêu

In [ ]:
for col in numerical_cols:
    viz.plot_numerical_vs_target(data, col)

**Nhận xét:**

**`city_development_index`:**
- Phân phối của nhóm `target=1` (muốn chuyển việc) có xu hướng tập trung ở các giá trị `city_development_index` **thấp hơn**. Điều này cho thấy ứng viên từ các thành phố kém phát triển hơn có xu hướng tìm việc mới cao hơn.

**`training_hours`:**
- Phân phối của `training_hours` giữa hai nhóm `target=0` và `target=1` **gần như giống hệt nhau**. Biến này dường như không phải là một yếu tố phân biệt mạnh mẽ.

#### Đánh giá sự khác biệt về thang đo (Scaling)

In [ ]:
viz.plot_scaling_comparison(data, 'city_development_index', 'training_hours')

**Nhận xét:**

Có sự khác biệt rất lớn về thang đo giữa `city_development_index` (0.4-1.0) và `training_hours` (1-336). Điều này bắt buộc phải thực hiện **scaling** (chuẩn hóa) cho các biến số trước khi đưa vào các mô hình nhạy cảm với khoảng cách như Logistic Regression, SVM, hoặc KNN.

### 4.2. Bảng Chéo (Cross-tabulations)

#### Biến phân loại vs. Biến phân loại

In [ ]:
# Lấy các giá trị duy nhất cho mỗi chiều
edu_levels = np.unique(data['education_level'])
exp_levels = np.unique(data['relevent_experience'])

# Tạo một ma trận rỗng để lưu số đếm
crosstab_counts = np.zeros((len(edu_levels), len(exp_levels)), dtype=int)

# Điền vào ma trận
for i, edu in enumerate(edu_levels):
    for j, exp in enumerate(exp_levels):
        # Tạo một mặt nạ boolean để đếm các lần xuất hiện
        mask = (data['education_level'] == edu) & (data['relevent_experience'] == exp)
        crosstab_counts[i, j] = np.sum(mask)

# Trực quan hóa bảng chéo bằng heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(crosstab_counts, annot=True, fmt='d', cmap='YlGnBu', 
            xticklabels=exp_levels, yticklabels=edu_levels)
plt.title('Heatmap: Education Level vs. Relevant Experience')
plt.ylabel('Education Level')
plt.xlabel('Relevant Experience')
plt.show()

**Nhận xét:**
Heatmap cho thấy phần lớn ứng viên có kinh nghiệm liên quan (`Has relevent experience`) là những người đã tốt nghiệp đại học (`Graduate`). Điều này là một mối quan hệ hợp lý và được mong đợi.

## 5. Quan Sát Ban Đầu & Insights

### Tóm tắt các quan sát chính

1.  **Việc thiếu dữ liệu là một tín hiệu mạnh:** Việc không cung cấp thông tin về công ty (`company_size`, `company_type`) có mối tương quan mạnh với việc một người đang tìm kiếm công việc mới.
2.  **Kinh nghiệm là yếu tố quyết định:** Kinh nghiệm làm việc càng ít, xu hướng tìm việc mới càng cao. Những người có dưới 5 năm kinh nghiệm đặc biệt có khả năng "nhảy việc" cao.
3.  **Môi trường làm việc quan trọng:** Nhân viên tại các công ty nhỏ và startup có tỷ lệ tìm việc mới cao hơn đáng kể so với các công ty lớn và ổn định hơn.
4.  **Vị trí địa lý có ảnh hưởng:** Ứng viên từ các thành phố có chỉ số phát triển thấp hơn có nhiều khả năng tìm kiếm cơ hội mới hơn.
5.  **Dữ liệu bị mất cân bằng:** Lớp `target=1` (tìm việc) chỉ chiếm khoảng 25% tập dữ liệu, điều này cần được xử lý trong giai đoạn mô hình hóa.

### Các vấn đề về chất lượng dữ liệu

- **Giá trị thiếu:** Nhiều cột có tỷ lệ thiếu đáng kể, cần có chiến lược xử lý thông minh (ví dụ: coi là một loại riêng) thay vì chỉ điền giá trị.
- **Cardinality cao:** Cột `city` có quá nhiều giá trị duy nhất, cần một chiến lược mã hóa đặc biệt (ví dụ: target encoding, frequency encoding) thay vì one-hot encoding.
- **Thang đo khác biệt:** Các biến số `city_development_index` và `training_hours` có thang đo rất khác nhau và cần được chuẩn hóa (scaling).

### Các bước tiền xử lý cần thiết

1.  **Xử lý giá trị thiếu:** Chuyển đổi các giá trị thiếu trong các cột phân loại thành một loại mới (ví dụ: 'Missing').
2.  **Mã hóa biến phân loại:**
    - Sử dụng One-Hot Encoding cho các biến có ít loại.
    - Sử dụng Target Encoding hoặc Frequency Encoding cho cột `city`.
    - Chuyển đổi các biến thứ tự (ordinal) như `experience` và `education_level` thành số.
3.  **Chuẩn hóa biến số:** Áp dụng `StandardScaler` hoặc `MinMaxScaler` cho các biến số.
4.  **Xử lý mất cân bằng:** Áp dụng SMOTE cho tập huấn luyện hoặc sử dụng `class_weight` trong mô hình.

### Cờ đỏ (Red Flags) & Hạn chế

- **Định nghĩa "tìm việc":** Dữ liệu không phân biệt giữa người đang tích cực tìm việc và người chỉ "thụ động" xem xét các cơ hội. Điều này có thể ảnh hưởng đến độ chính xác của mô hình.
- **Thiên vị lựa chọn (Selection Bias):** Dữ liệu chỉ bao gồm những người đã tham gia một khóa đào tạo. Kết quả có thể không khái quát hóa được cho toàn bộ lực lượng lao động.